# Market Predictor: Backtesting

This notebook implements comprehensive backtesting of our trading strategy:
1. Strategy Implementation
2. Historical Simulation
3. Performance Analysis
4. Risk Assessment
5. Robustness Testing

## Setup and Configuration

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Add project root to path
sys.path.append('..')

# Import project modules
from src.models import load_ensemble_model
from src.utils import (
    setup_project_logger,
    ModelMetrics,
    TradingMetrics,
    RiskMetrics
)
from config import Config, load_validated_config

# Import required libraries
import joblib
import empyrical as ep
import pyfolio as pf
from scipy import stats

# Plotting settings
plt.style.use('seaborn')
%matplotlib inline
sns.set_theme(style="whitegrid")

# Setup logging
logger = setup_project_logger('backtesting')

## 1. Loading Data and Model

Load the trained model and historical data for backtesting:
- Load final ensemble model
- Prepare historical market data
- Set up backtesting parameters

In [ ]:
# Load configuration
config = load_validated_config('config/parameters.yaml')

# Load model and scaler
model = joblib.load('models/final_ensemble.joblib')
scaler = joblib.load('models/feature_scaler.joblib')

# Load historical data
market_data = pd.read_parquet('data/processed/market_data.parquet')
features_data = pd.read_parquet('data/features/selected_features.parquet')

# Load feature metadata
with open('data/features/feature_metadata.json', 'r') as f:
    feature_metadata = json.load(f)

# Set up backtesting parameters
backtest_config = {
    'initial_capital': 100000,
    'position_size': 0.1,  # 10% of portfolio per trade
    'stop_loss': 0.02,     # 2% stop loss
    'take_profit': 0.03,   # 3% take profit
    'max_positions': 5,    # Maximum number of simultaneous positions
    'transaction_costs': 0.001  # 0.1% transaction cost
}

# Create backtesting time periods
test_start = config.data.test_start
test_period = market_data.loc[test_start:]
print("\nBacktesting Period:")
print(f"Start: {test_period.index[0].strftime('%Y-%m-%d')}")
print(f"End: {test_period.index[-1].strftime('%Y-%m-%d')}")
print(f"Total trading days: {len(test_period)}")

# Plot data overview
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Price plot
market_data['Close'].plot(ax=ax1)
ax1.axvline(test_start, color='r', linestyle='--', label='Test Start')
ax1.set_title('Market Price')
ax1.legend()

# Volume plot
market_data['Volume'].plot(ax=ax2)
ax2.axvline(test_start, color='r', linestyle='--', label='Test Start')
ax2.set_title('Trading Volume')
ax2.legend()

plt.tight_layout()
plt.show()

logger.info('Data and model loaded successfully')

## 2. Strategy Implementation

Implement the trading strategy with position management:
- Signal generation
- Position sizing
- Risk management
- Order execution simulation

In [ ]:
class TradingStrategy:
    def __init__(self, model, scaler, config):
        self.model = model
        self.scaler = scaler
        self.config = config
        self.positions = {}  # Current open positions
        self.capital = config['initial_capital']
        self.available_capital = config['initial_capital']
        
    def generate_signals(self, features: pd.DataFrame) -> pd.Series:
        """Generate trading signals from model predictions"""
        # Scale features
        scaled_features = self.scaler.transform(features)
        
        # Get predictions
        probabilities = self.model.predict_proba(scaled_features)
        
        # Convert to signals (1: Long, 0: No position)
        signals = (probabilities[:, 1] > 0.5).astype(int)
        
        return pd.Series(signals, index=features.index)
    
    def calculate_position_size(self, price: float) -> float:
        """Calculate position size based on available capital"""
        max_position_size = self.available_capital * self.config['position_size']
        num_shares = np.floor(max_position_size / price)
        return num_shares
    
    def simulate_trades(self, market_data: pd.DataFrame, features: pd.DataFrame) -> pd.DataFrame:
        """Simulate trading with position management"""
        signals = self.generate_signals(features)
        trades = []
        daily_stats = []
        
        for date in market_data.index:
            stats = self._process_trading_day(
                date,
                market_data.loc[date],
                signals.loc[date]
            )
            daily_stats.append(stats)
        
        return pd.DataFrame(daily_stats, index=market_data.index)
    
    def _process_trading_day(self, date, market_data, signal) -> dict:
        """Process single trading day"""
        # Update existing positions
        self._update_positions(date, market_data)
        
        # Process new signal
        if signal == 1 and len(self.positions) < self.config['max_positions']:
            self._open_position(date, market_data)
        
        # Calculate daily statistics
        stats = self._calculate_daily_stats(date, market_data)
        
        return stats
    
    def _update_positions(self, date, market_data):
        """Update existing positions with stops and targets"""
        closed_positions = []
        
        for entry_date, position in self.positions.items():
            # Check stop loss
            if market_data['Low'] <= position['stop_price']:
                position['exit_price'] = position['stop_price']
                position['exit_date'] = date
                position['exit_type'] = 'stop_loss'
                closed_positions.append(entry_date)
                
            # Check take profit
            elif market_data['High'] >= position['target_price']:
                position['exit_price'] = position['target_price']
                position['exit_date'] = date
                position['exit_type'] = 'take_profit'
                closed_positions.append(entry_date)
        
        # Close positions and update capital
        for entry_date in closed_positions:
            position = self.positions[entry_date]
            pnl = (position['exit_price'] - position['entry_price']) * position['shares']
            transaction_cost = (position['entry_price'] + position['exit_price']) * \
                             position['shares'] * self.config['transaction_costs']
            
            self.capital += (pnl - transaction_cost)
            self.available_capital += (position['entry_price'] * position['shares'])
            del self.positions[entry_date]
    
    def _open_position(self, date, market_data):
        """Open new position"""
        entry_price = market_data['Close']
        shares = self.calculate_position_size(entry_price)
        
        if shares > 0:
            self.positions[date] = {
                'entry_price': entry_price,
                'shares': shares,
                'stop_price': entry_price * (1 - self.config['stop_loss']),
                'target_price': entry_price * (1 + self.config['take_profit']),
                'entry_date': date
            }
            self.available_capital -= (entry_price * shares)
    
    def _calculate_daily_stats(self, date, market_data) -> dict:
        """Calculate daily portfolio statistics"""
        position_value = sum(
            market_data['Close'] * position['shares']
            for position in self.positions.values()
        )
        
        return {
            'date': date,
            'capital': self.capital,
            'position_value': position_value,
            'total_value': self.capital + position_value,
            'num_positions': len(self.positions),
            'available_capital': self.available_capital
        }

# Initialize strategy
strategy = TradingStrategy(model, scaler, backtest_config)

# Run backtest simulation
backtest_results = strategy.simulate_trades(test_period, features_data.loc[test_period.index])

# Plot equity curve
plt.figure(figsize=(15, 7))
backtest_results['total_value'].plot(label='Portfolio Value')
plt.title('Portfolio Equity Curve')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True)
plt.show()

logger.info('Strategy simulation completed')

## 3. Performance Analysis

Analyze the backtesting results:
- Return metrics
- Risk metrics
- Trade statistics
- Performance visualization

In [ ]:
# Calculate performance metrics
class BacktestAnalysis:
    def __init__(self, results: pd.DataFrame, market_data: pd.DataFrame):
        self.results = results
        self.market_data = market_data
        
    def calculate_returns(self) -> pd.DataFrame:
        """Calculate strategy and benchmark returns"""
        # Strategy returns
        strategy_returns = self.results['total_value'].pct_change()
        
        # Benchmark returns (market)
        benchmark_returns = self.market_data['Close'].pct_change()
        
        return pd.DataFrame({
            'strategy': strategy_returns,
            'benchmark': benchmark_returns
        })
    
    def calculate_metrics(self) -> dict:
        """Calculate performance metrics"""
        returns = self.calculate_returns()
        
        metrics = {
            'Total Return (%)': (self.results['total_value'].iloc[-1] / 
                               self.results['total_value'].iloc[0] - 1) * 100,
            'Annual Return (%)': ep.annual_return(returns['strategy']) * 100,
            'Annual Volatility (%)': ep.annual_volatility(returns['strategy']) * 100,
            'Sharpe Ratio': ep.sharpe_ratio(returns['strategy']),
            'Sortino Ratio': ep.sortino_ratio(returns['strategy']),
            'Max Drawdown (%)': ep.max_drawdown(returns['strategy']) * 100,
            'Calmar Ratio': ep.calmar_ratio(returns['strategy']),
            'Alpha': ep.alpha(returns['strategy'], returns['benchmark']),
            'Beta': ep.beta(returns['strategy'], returns['benchmark'])
        }
        
        return metrics

# Initialize analysis
analysis = BacktestAnalysis(backtest_results, test_period)
returns = analysis.calculate_returns()
metrics = analysis.calculate_metrics()

# Print performance metrics
print("\nPerformance Metrics:")
print("=" * 50)
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")

# Create performance visualizations
fig = plt.figure(figsize=(15, 15))

# 1. Cumulative Returns
plt.subplot(3, 2, 1)
(1 + returns).cumprod().plot()
plt.title('Cumulative Returns')
plt.grid(True)

# 2. Returns Distribution
plt.subplot(3, 2, 2)
returns['strategy'].hist(bins=50, alpha=0.5, label='Strategy')
returns['benchmark'].hist(bins=50, alpha=0.5, label='Benchmark')
plt.title('Returns Distribution')
plt.legend()

# 3. Rolling Sharpe Ratio
plt.subplot(3, 2, 3)
rolling_sharpe = returns['strategy'].rolling(window=252).apply(
    lambda x: ep.sharpe_ratio(x)
)
rolling_sharpe.plot()
plt.title('Rolling Sharpe Ratio (1 Year)')
plt.grid(True)

# 4. Drawdown
plt.subplot(3, 2, 4)
ep.cum_returns(returns['strategy']).plot()
plt.title('Drawdown')
plt.grid(True)

# 5. Rolling Beta
plt.subplot(3, 2, 5)
rolling_beta = returns.rolling(window=252).apply(
    lambda x: ep.beta(x['strategy'], x['benchmark'])
)
rolling_beta.plot()
plt.title('Rolling Beta (1 Year)')
plt.grid(True)

# 6. Rolling Correlation
plt.subplot(3, 2, 6)
rolling_corr = returns['strategy'].rolling(window=252).corr(returns['benchmark'])
rolling_corr.plot()
plt.title('Rolling Correlation with Benchmark (1 Year)')
plt.grid(True)

plt.tight_layout()
plt.show()

logger.info('Performance analysis completed')

## 4. Risk Analysis

Detailed analysis of strategy risks:
- Value at Risk (VaR)
- Expected Shortfall
- Stress testing
- Risk factor analysis

In [ ]:
class RiskAnalysis:
    def __init__(self, returns: pd.Series, positions: pd.DataFrame):
        self.returns = returns
        self.positions = positions
        
    def calculate_var(self, confidence_level: float = 0.95) -> dict:
        """Calculate Value at Risk metrics"""
        # Historical VaR
        hist_var = np.percentile(self.returns, (1 - confidence_level) * 100)
        
        # Parametric VaR
        mean = self.returns.mean()
        std = self.returns.std()
        param_var = stats.norm.ppf(1 - confidence_level, mean, std)
        
        # Conditional VaR (Expected Shortfall)
        cvar = self.returns[self.returns <= hist_var].mean()
        
        return {
            'Historical VaR': hist_var,
            'Parametric VaR': param_var,
            'Conditional VaR': cvar
        }
    
    def stress_test(self, scenarios: dict) -> pd.DataFrame:
        """Perform stress testing under different scenarios"""
        results = {}
        
        for scenario_name, shock in scenarios.items():
            # Apply shock to returns
            stressed_returns = self.returns * (1 + shock)
            
            # Calculate metrics under stress
            results[scenario_name] = {
                'Mean Return': stressed_returns.mean(),
                'Volatility': stressed_returns.std(),
                'Max Drawdown': ep.max_drawdown(stressed_returns),
                'Sharpe Ratio': ep.sharpe_ratio(stressed_returns)
            }
            
        return pd.DataFrame(results).T
    
    def calculate_risk_metrics(self) -> dict:
        """Calculate comprehensive risk metrics"""
        return {
            'Daily Volatility': self.returns.std(),
            'Annualized Volatility': self.returns.std() * np.sqrt(252),
            'Skewness': self.returns.skew(),
            'Kurtosis': self.returns.kurtosis(),
            'Worst Day': self.returns.min(),
            'Best Day': self.returns.max(),
            'Days to Recovery': self._calculate_recovery_time()
        }
    
    def _calculate_recovery_time(self) -> float:
        """Calculate average recovery time from drawdowns"""
        cum_returns = (1 + self.returns).cumprod()
        running_max = cum_returns.expanding().max()
        drawdowns = cum_returns / running_max - 1
        
        # Find recovery periods
        in_drawdown = False
        drawdown_start = None
        recovery_times = []
        
        for date, value in drawdowns.items():
            if value < 0 and not in_drawdown:
                drawdown_start = date
                in_drawdown = True
            elif value >= 0 and in_drawdown:
                recovery_time = (date - drawdown_start).days
                recovery_times.append(recovery_time)
                in_drawdown = False
                
        return np.mean(recovery_times) if recovery_times else 0

# Initialize risk analysis
risk_analyzer = RiskAnalysis(returns['strategy'], backtest_results)

# Calculate VaR metrics
var_metrics = risk_analyzer.calculate_var()
print("\nValue at Risk Metrics (95% Confidence):")
print("=" * 50)
for metric, value in var_metrics.items():
    print(f"{metric}: {value:.4f}")

# Perform stress tests
stress_scenarios = {
    'Market Crash': -0.15,      # -15% shock
    'Volatility Spike': 0.50,   # +50% volatility
    'Moderate Decline': -0.05,  # -5% shock
    'Strong Rally': 0.10        # +10% shock
}

stress_results = risk_analyzer.stress_test(stress_scenarios)
print("\nStress Test Results:")
print("=" * 50)
print(stress_results)

# Calculate risk metrics
risk_metrics = risk_analyzer.calculate_risk_metrics()
print("\nRisk Metrics:")
print("=" * 50)
for metric, value in risk_metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot risk visualizations
fig = plt.figure(figsize=(15, 10))

# 1. Rolling VaR
plt.subplot(2, 2, 1)
rolling_var = returns['strategy'].rolling(window=252).apply(
    lambda x: np.percentile(x, 5)
)
rolling_var.plot(label='VaR')
plt.title('Rolling 95% VaR (1 Year)')
plt.grid(True)

# 2. Return Distribution with VaR
plt.subplot(2, 2, 2)
returns['strategy'].hist(bins=50, density=True, alpha=0.7)
plt.axvline(var_metrics['Historical VaR'], color='r', linestyle='--', 
            label='Historical VaR')
plt.axvline(var_metrics['Parametric VaR'], color='g', linestyle='--', 
            label='Parametric VaR')
plt.title('Return Distribution with VaR')
plt.legend()

# 3. Drawdown Analysis
plt.subplot(2, 2, 3)
cum_returns = (1 + returns['strategy']).cumprod()
running_max = cum_returns.expanding().max()
drawdowns = (cum_returns / running_max - 1)
drawdowns.plot()
plt.title('Drawdown Analysis')
plt.grid(True)

# 4. Risk Contributions
plt.subplot(2, 2, 4)
risk_contribution = pd.Series(risk_metrics).sort_values()
risk_contribution.plot(kind='bar')
plt.title('Risk Metric Contributions')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

logger.info('Risk analysis completed')

## 5. Robustness Testing and Conclusions

Test strategy robustness across different conditions and summarize findings:
- Parameter sensitivity
- Market regime analysis
- Performance consistency
- Final recommendations

In [ ]:
# Parameter Sensitivity Analysis
def parameter_sensitivity_test(base_config: dict, param_ranges: dict) -> pd.DataFrame:
    """Test strategy performance across parameter ranges"""
    sensitivity_results = []
    
    for param, values in param_ranges.items():
        for value in values:
            # Create modified config
            test_config = base_config.copy()
            test_config[param] = value
            
            # Run strategy with modified parameter
            test_strategy = TradingStrategy(model, scaler, test_config)
            results = test_strategy.simulate_trades(test_period, features_data.loc[test_period.index])
            
            # Calculate key metrics
            returns = results['total_value'].pct_change()
            metrics = {
                'Parameter': param,
                'Value': value,
                'Sharpe Ratio': ep.sharpe_ratio(returns),
                'Total Return': (results['total_value'].iloc[-1] / 
                               results['total_value'].iloc[0] - 1),
                'Max Drawdown': ep.max_drawdown(returns)
            }
            sensitivity_results.append(metrics)
    
    return pd.DataFrame(sensitivity_results)

# Define parameter ranges to test
param_ranges = {
    'position_size': [0.05, 0.1, 0.15, 0.2],
    'stop_loss': [0.01, 0.02, 0.03, 0.04],
    'take_profit': [0.02, 0.03, 0.04, 0.05],
    'max_positions': [3, 4, 5, 6]
}

# Run sensitivity analysis
sensitivity_results = parameter_sensitivity_test(backtest_config, param_ranges)

# Market Regime Analysis
def analyze_market_regimes(returns: pd.Series, market_returns: pd.Series) -> pd.DataFrame:
    """Analyze strategy performance in different market regimes"""
    # Define market regimes
    regimes = pd.qcut(market_returns.rolling(20).mean(), q=3, labels=['Bear', 'Neutral', 'Bull'])
    
    regime_performance = []
    for regime in ['Bear', 'Neutral', 'Bull']:
        regime_returns = returns[regimes == regime]
        
        metrics = {
            'Regime': regime,
            'Average Return': regime_returns.mean(),
            'Volatility': regime_returns.std(),
            'Sharpe Ratio': ep.sharpe_ratio(regime_returns),
            'Win Rate': (regime_returns > 0).mean()
        }
        regime_performance.append(metrics)
    
    return pd.DataFrame(regime_performance)

# Run market regime analysis
regime_analysis = analyze_market_regimes(returns['strategy'], returns['benchmark'])

# Plot analysis results
fig = plt.figure(figsize=(15, 10))

# 1. Parameter Sensitivity
plt.subplot(2, 2, 1)
for param in param_ranges.keys():
    param_data = sensitivity_results[sensitivity_results['Parameter'] == param]
    plt.plot(param_data['Value'], param_data['Sharpe Ratio'], 
             marker='o', label=param)
plt.title('Parameter Sensitivity - Sharpe Ratio')
plt.legend()
plt.grid(True)

# 2. Market Regime Performance
plt.subplot(2, 2, 2)
regime_analysis['Sharpe Ratio'].plot(kind='bar')
plt.title('Performance by Market Regime')
plt.grid(True)

# 3. Rolling Window Analysis
plt.subplot(2, 2, 3)
windows = [30, 60, 90]
for window in windows:
    rolling_sharpe = returns['strategy'].rolling(window).apply(
        lambda x: ep.sharpe_ratio(x)
    )
    rolling_sharpe.plot(label=f'{window}d')
plt.title('Rolling Sharpe Ratio - Multiple Windows')
plt.legend()
plt.grid(True)

# 4. Performance Consistency
plt.subplot(2, 2, 4)
monthly_returns = returns['strategy'].resample('M').sum()
monthly_returns.hist(bins=20)
plt.title('Distribution of Monthly Returns')
plt.grid(True)

plt.tight_layout()
plt.show()

# Print conclusions
print("\nStrategy Analysis Conclusions:")
print("=" * 50)
print("\n1. Parameter Sensitivity:")
print(f"Most sensitive parameter: {sensitivity_results.groupby('Parameter')['Sharpe Ratio'].std().idxmax()}")
print("\n2. Market Regime Performance:")
print(regime_analysis)
print("\n3. Recommendations:")
print("- Optimal position size:", sensitivity_results.loc[sensitivity_results['Sharpe Ratio'].idxmax(), 'Value'])
print("- Best performing market regime:", regime_analysis.loc[regime_analysis['Sharpe Ratio'].idxmax(), 'Regime'])
print("- Average recovery time:", risk_metrics['Days to Recovery'], "days")

logger.info('Robustness testing completed')

# Save final analysis results
final_analysis = {
    'sensitivity_analysis': sensitivity_results.to_dict(),
    'regime_analysis': regime_analysis.to_dict(),
    'risk_metrics': risk_metrics,
    'performance_metrics': metrics
}

with open('results/final_analysis.json', 'w') as f:
    json.dump(final_analysis, f, indent=4)